In [1]:
import os
import numpy as np

import CAS.dmrghandler.src.dmrghandler as dmrghandler
from CAS.dmrghandler.src.dmrghandler import dmrg_calc_prepare
from pathlib import Path
import pyscf.tools.fcidump
import pyscf
import scipy

In [2]:
def rename_files(directory):
    """
    Rename the files which have unsupported characters
    Args:
        directory: 

    Returns:

    """
    # List all files in the given directory
    files = os.listdir(directory)
    # Sort the files to ensure consistent order
    files.sort()
    
    for index, filename in enumerate(files):
        # Construct the old file path
        old_path = os.path.join(directory, filename)
        
        # Ensure we only rename files, not directories
        if os.path.isfile(old_path):
            # Replace slashes with underscores
            new_filename = filename.replace('/', '_')
            new_filename = new_filename.replace(',', '')
            
            new_filename = new_filename.replace(':', '_')
            
            new_filename = new_filename.replace('+', '_')
            
            new_filename = new_filename.replace('*', '_')
            
            # Remove all spaces
            new_filename = new_filename.replace(' ', '')
            
            # Construct the new file path
            new_path = os.path.join(directory, new_filename)
            
            try:
                # Rename the file
                os.rename(old_path, new_path)
            except OSError as e:
                print(f'Error renaming {old_path} to {new_path}: {e}')

In [3]:
rename_files(Path("fcidumps_catalysts"))

In [4]:
fcidump_path ="fcidumps_catalysts"
fcidump_output_path = Path("fcidumps_LPBLISS_combined")
for filename in os.listdir(fcidump_path):
    print(f"Current file: {filename}")
    (
    one_body_tensor,
    two_body_tensor,
    nuc_rep_energy,
    num_orbitals,
    num_spin_orbitals,
    num_electrons,
    two_S,
    two_Sz,
    orb_sym,
    extra_attributes,
    ) = dmrg_calc_prepare.load_tensors_from_fcidump(data_file_path=Path(fcidump_path)/Path(filename), molpro_orbsym_convention=True)
    
    # Combine obt and tbt
    absorbed_tbt = pyscf.fci.direct_nosym.absorb_h1e(np.array(one_body_tensor), np.array(two_body_tensor), norb=num_orbitals, nelec=num_electrons)
    h_in_tbt = pyscf.ao2mo.restore(1, absorbed_tbt.copy(), num_orbitals).astype(np.array(one_body_tensor).dtype, copy=False)
    
    obt_dummy = np.zeros((num_orbitals, num_orbitals))
    
    label = "_tbt_combined"
    filename = filename + f"{label}"
    pyscf.tools.fcidump.from_integrals(
        fcidump_output_path/Path(filename),
        obt_dummy,
        h_in_tbt,
        nmo=obt_dummy.shape[0],
        nelec=num_electrons,
        nuc=nuc_rep_energy,
        ms=two_S,
        orbsym=None,
        tol=1E-8,
        float_format=' %.16g',
    )

Current file: fcidump.26_pc_{'Mo'_'def2-SVP'_'I'_'def2-SVP'_'Cl'_'def2-SVP'_'default'_'6-311_G(dp)'}
Parsing fcidumps_catalysts/fcidump.26_pc_{'Mo'_'def2-SVP'_'I'_'def2-SVP'_'Cl'_'def2-SVP'_'default'_'6-311_G(dp)'}
Current file: fcidump.8_melact_6-311__G__
Parsing fcidumps_catalysts/fcidump.8_melact_6-311__G__
Current file: fcidump.65_5_15_af_{'default'_'cc-pVTZ''Mn'_'lanl2tz'}
Parsing fcidumps_catalysts/fcidump.65_5_15_af_{'default'_'cc-pVTZ''Mn'_'lanl2tz'}
Current file: fcidump.21_rc_{'Mo'_'def2-SVP'_'I'_'def2-SVP'_'Cl'_'def2-SVP'_'default'_'6-311_G(dp)'}
Parsing fcidumps_catalysts/fcidump.21_rc_{'Mo'_'def2-SVP'_'I'_'def2-SVP'_'Cl'_'def2-SVP'_'default'_'6-311_G(dp)'}
Current file: fcidump.42_1_star_{'default'_'cc-pVTZ''Ru'_'LANL2TZ'}
Parsing fcidumps_catalysts/fcidump.42_1_star_{'default'_'cc-pVTZ''Ru'_'LANL2TZ'}
Current file: fcidump.38_1_ts_{'default'_'cc-pVTZ''Ru'_'LANL2TZ'}
Parsing fcidumps_catalysts/fcidump.38_1_ts_{'default'_'cc-pVTZ''Ru'_'LANL2TZ'}
Current file: fcidump.59_5_1